In [ ]:
import pandas as pd

df = pd.read_csv('MY2022 Fuel Consumption Ratings.csv')
dfe = pd.read_csv('csvveiculos.csv')

df.columns = ['Año Modelo','Marca','Modelo','Clase Vehiculo','Tamaño Maquina(L)','Cilindros','Transmicion',
'Tipo Fuel','Consumo Fuel(ciudad(L/100km)','Consumo Fuel(carretera(L/100km)','Consumo Fuel(Consumo Combinado(L/100km)',
'Consumo Fuel(Consumo Combinado(mpg(millas por galon))','CO2 emiciones(g/km)','CO2 Rating','Smog Rating']

df.groupby("Marca").mean()

In [ ]:
dfe

            +" ("+str("{:.2f}".format(distance_rec))+" km recorridos)"